In [1]:
import pandas as pd
import xopen
import json
from huggingface_hub import InferenceClient


# Inspired by 
https://python.langchain.com/docs/use_cases/question_answering/in_memory_question_answering

```python
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text translated into italian.
{context}
Question: {question}
Relevant text, if any, in Italian:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer italian.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

QUESTION: {question}
=========
{summaries}
=========
Answer in Italian:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)
``````

In [3]:
HUGGINGFACEHUB_API_TOKEN = "YOUR_API_TOKEN"
HF_HUB_MODELS = {
    "llama-2-70b": "meta-llama/Llama-2-70b-chat-hf",
    "llama-2-13b": "meta-llama/Llama-2-13b-chat-hf",
    "llama-2-7b": "meta-llama/Llama-2-7b-chat-hf",
}

In [6]:
docs = """"Document [1](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad R\u00f6ntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice\u2014in 1956 and 1972. Maria Sk\u0142odowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded\nDocument [2](Title: Asian Americans in science and technology) Prize in physics for discovery of the subatomic particle J/\u03c8. Subrahmanyan Chandrasekhar shared the 1983 Nobel Prize in Physics and had the Chandra X-ray Observatory named after him. Steven Chu shared the 1997 Nobel Prize in Physics for his research in cooling and trapping atoms using laser light. Daniel Tsui shared the 1998 Nobel Prize in Physics in 1998 for helping discover the fractional Quantum Hall effect. Yoichiro Nambu received the 2008 Nobel Prize in Physics for his work on the consequences of spontaneously broken symmetries in field theories. In 2009, Charles K. Kao was awarded Nobel Prize in Physics\nDocument [3](Title: Scientist) and pursued through a unique method, was essentially in place. Ram\u00f3n y Cajal won the Nobel Prize in 1906 for his remarkable observations in neuroanatomy. Marie Curie became the first female to win the Nobel Prize and the first person to win it twice. Her efforts led to the development of nuclear energy and Radio therapy for the treatment of cancer. In 1922, she was appointed a member of the International Commission on Intellectual Co-operation by the Council of the League of Nations. She campaigned for scientist's right to patent their discoveries and inventions. She also campaigned for free access\nDocument [4](Title: Norwegian Americans) science, Ernest Lawrence won the Nobel Prize in Physics in 1939. Lars Onsager won the 1968 Nobel Prize in Chemistry. Norman Borlaug, father of the Green Revolution, won the Nobel Peace Prize in 1970. Christian B. Anfinsen won the Nobel Prize for chemistry in 1972. Ivar Giaever won the Nobel Prize in Physics 1973. Carl Richard Hagen is noted for his work in physics. In engineering, Clayton Jacobson II is credited with the invention of the modern personal watercraft. Ole Singstad was a pioneer of underwater tunnels. Ole Evinrude invented the first outboard motor with practical commercial application, recognizable today\nDocument [5](Title: Maria Goeppert Mayer) of US postage stamps, along with Melvin Calvin, Asa Gray, and Severo Ochoa. Her papers are in the Geisel Library at the University of California, San Diego, and the university's physics department is housed in Mayer Hall, which is named after her and her husband. Maria Goeppert Mayer Maria Goeppert Mayer (June 28, 1906 \u2013 February 20, 1972) was a German-born American theoretical physicist, and Nobel laureate in Physics for proposing the nuclear shell model of the atomic nucleus. She was the second woman to win a Nobel Prize in physics, the first being Marie Curie. A graduate of the\nDocument [6](Title: University of Chicago) Medal, which is rewarded annually to the best economist under the age of 40, has also been awarded to 4 current members of the university faculty. Notable faculty in physics have included the speed of light calculator A. A. Michelson, elementary charge calculator Robert A. Millikan, discoverer of the Compton Effect Arthur H. Compton, the creator of the first nuclear reactor Enrico Fermi, \"the father of the hydrogen bomb\" Edward Teller, \"one of the most brilliant and productive experimental physicists of the twentieth century\" Luis Walter Alvarez, Murray Gell-Mann who introduced the quark, second female Nobel laureate Maria Goeppert-Mayer, the\nDocument [7](Title: E\u0301cole normale supe\u0301rieure (Paris)) was also awarded the Abel prize. In addition, eight \"normaliens\" have gone on to receive the Nobel Prize in Physics: Claude Cohen-Tannoudji, Pierre-Gilles de Gennes, Albert Fert, Alfred Kastler, Gabriel Lippmann, Louis N\u00e9el, Jean Baptiste Perrin and Serge Haroche, while other ENS physicists include such major figures as Paul Langevin, famous for developing Langevin dynamics and the Langevin equation. Alumnus Paul Sabatier won the Nobel Prize in Chemistry. A ranking of universities worldwide based on ratios of alumni to Nobel prize-winners published in 2016 by American scholars Stephen Hsu and Jonathan Wai placed ENS as the first university worldwide, far\nDocument [8](Title: Marie Curie) named in her honor. Marie Curie Marie Sk\u0142odowska Curie (; ; ; born Maria Salomea Sk\u0142odowska; 7 November 18674 July 1934) was a Polish and naturalized-French physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize, the first person and only woman to win twice, the only person to win a Nobel Prize in two different sciences, and was part of the Curie family legacy of five Nobel Prizes. She was also the first woman to become a professor at the University of Paris, and in 1995 became the first woman\nDocument [9](Title: St John's College, Cambridge) Nobel Prize winners: Sir Edward Appleton, for discovering the Appleton layer, Sir John Cockcroft KCB, physicist who first split the atom, Allan Cormack, for the invention of the CAT scan, Paul Dirac, one of the founders of quantum mechanics, Sir Nevill Francis Mott, for work on the behaviour of electrons in magnetic solids, Abdus Salam, for unifying the electromagnetic force and the weak force, Frederick Sanger, molecular biologist, Maurice Wilkins, awarded Nobel prize for Medicine or Physiology with Watson and Crick for discovering the structure of DNA, and Eric Maskin (visiting & honorary fellow), awarded the Nobel Memorial Prize in\nDocument [10](Title: Arthur H. Rosenfeld) Nobel Prize-winning Italian physicist. Rosenfeld coauthored a book on nuclear physics with Fermi, who was noted for building the world's first nuclear reactor. In 1954, after earning his PhD in physics, and with a recommendation from Fermi, Rosenfeld accepted a position as a teaching physicist at the University of California, Berkeley. At Berkeley, Rosenfeld joined the University of California Department of Physics and the particle physics research group at Lawrence Berkeley National Lab (LBNL) led by Nobel Laureate Luis Alvarez. Alvarez went on to win the Nobel Prize with research backed by his team of scientists that included Rosenfeld. In\nDocument [11](Title: Japanese Americans) J. Pedersen won the 1987 Nobel Prize in Chemistry for his methods of synthesizing crown ethers. Yoichiro Nambu won the 2008 Nobel Prize in Physics for his work on quantum chromodynamics and spontaneous symmetry breaking. Shuji Nakamura won the 2014 Nobel Prize in Physics for the invention of efficient blue light-emitting diodes. Michio Kaku is a theoretical physicist specializing in string field theory, and a well-known science popularizer. Ellison Onizuka became the first Asian American astronaut and was the mission specialist aboard \"Challenger\" at the time of its explosion. Immunologist Santa J. Ono became the first Japanese American president of\nDocument [12](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was\nDocument [13](Title: Paris in the Belle E\u0301poque) studying the fluorescence of uranium salts, discovered radioactivity in 1896, and in 1903 was awarded the Nobel Prize in physics for his discovery. Pierre Curie (1859-1906) and Marie Curie (1867-1934) jointly carried on Becquerel's work, discovering radium and polonium (1898). They jointly received the Nobel Prize for physics in 1903. Marie Curie became the first female professor at the University of Paris and won the Nobel Prize for chemistry in 1911. She was the first woman to be buried in the Panth\u00e9on. The neon light was used for the first time in Paris on 3 December 1910 in the Grand\nDocument [14](Title: Richard Feynman) by Michael Gottlieb and Ralph Leighton (Robert Leighton's son), with support from Kip Thorne and other physicists. Richard Feynman Richard Phillips Feynman (; May 11, 1918 \u2013 February 15, 1988) was an American theoretical physicist, known for his work in the path integral formulation of quantum mechanics, the theory of quantum electrodynamics, and the physics of the superfluidity of supercooled liquid helium, as well as in particle physics for which he proposed the parton model. For his contributions to the development of quantum electrodynamics, Feynman, jointly with Julian Schwinger and Shin'ichir\u014d Tomonaga, received the Nobel Prize in Physics in 1965.\nDocument [15](Title: Marie Curie) the Nobel Prize in Physics, \"in recognition of the extraordinary services they have rendered by their joint researches on the radiation phenomena discovered by Professor Henri Becquerel.\" At first the committee had intended to honor only Pierre Curie and Henri Becquerel, but a committee member and advocate for women scientists, Swedish mathematician Magnus Goesta Mittag-Leffler, alerted Pierre to the situation, and after his complaint, Marie's name was added to the nomination. Marie Curie was the first woman to be awarded a Nobel Prize. Curie and her husband declined to go to Stockholm to receive the prize in person; they were\nDocument [16](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the \"Hindustan Times\", \"The 2005 Nobel prize for Physics was awarded for my work, but I wasn't the one to get it. Each one of the discoveries that this Nobel was given for work based on my research.\" Sudarshan also commented on not being selected for the 1979 Nobel, \"Steven Weinberg, Sheldon Glashow and Abdus Salam built on work I had done as a 26-year-old student. If you give a prize for a building, shouldn\u2019t the fellow who built the first floor be given the prize before those who built the second\nDocument [17](Title: Rosalyn Sussman Yalow) the National Medal of Science, which is given to American individuals who deserve the highest honor in science and technology. In 1993, Yalow was inducted into the National Women's Hall of Fame. Rosalyn Sussman Yalow Rosalyn Sussman Yalow (July 19, 1921 \u2013 May 30, 2011) was an American medical physicist, and a co-winner of the 1977 Nobel Prize in Physiology or Medicine (together with Roger Guillemin and Andrew Schally) for development of the radioimmunoassay (RIA) technique. She was the second woman (the first being Gerty Cori), and the first American-born woman, to be awarded the Nobel Prize in physiology or\nDocument [18](Title: Poles) and chemist who conducted pioneering research on radioactivity and was the first woman to win a Nobel Prize, the first person and only woman to win twice, the only person to win twice in multiple sciences, and was part of the Curie family legacy of five Nobel Prizes. Another notable Polish expatriate scientist was Ignacy Domeyko (1802\u201389), a geologist and mineralogist who lived and worked in South America, in Chile. Kazimierz Funk (1884\u20131967), whose name is commonly anglicized as \"Casimir Funk\", was a Polish biochemist, generally credited with being among the first to formulate (in 1912) the concept of vitamins,\nDocument [19](Title: Nobel Prize) His son, George Paget Thomson, received the same prize in 1937 for showing that they also have the properties of waves. William Henry Bragg and his son, William Lawrence Bragg, shared the Physics Prize in 1915 for inventing the X-ray spectrometer. Niels Bohr was awarded the Physics prize in 1922, as was his son, Aage Bohr, in 1975. Manne Siegbahn, who received the Physics Prize in 1924, was the father of Kai Siegbahn, who received the Physics Prize in 1981. Hans von Euler-Chelpin, who received the Chemistry Prize in 1929, was the father of Ulf von Euler, who was awarded\nDocument [20](Title: Charles Russell Bardeen) business before marrying, and was an active figure in the art world. After her death from cancer in 1921, Charles married Ruth Hames. His son, Dr. John Bardeen, became the only person to win the Nobel Prize in Physics twice, in 1956 and 1972. Bardeen died in Madison, Wisconsin in 1935, from pancreatic cancer. He was succeeded as Dean of the University of Wisconsin Medical School by Dr. William Shainline Middleton. Charles Russell Bardeen Charles Russell Bardeen (8 February 1871 \u2013 12 June 1935) was an American physician and anatomist and the first dean of the University of Wisconsin Medical\n\n"""
q = "who got the first nobel prize in physics"

In [13]:
question_template = """<s>[INST] <<SYS>>
Use the following documents to see if any of the text is relevant to answer the question.
<</SYS>>

{search_results}
Question: {question}
Relevant text, if any:[/INST]"""

In [14]:
print(question_template.format(search_results=docs, question=q))

<s>[INST] <<SYS>>
Use the following documents to see if any of the text is relevant to answer the question.
<</SYS>>

"Document [1](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice—in 1956 and 1972. Maria Skłodowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded
Document [2](Title: Asian Americans in science and technology) Prize in physics for discovery of the subatomic particle J/ψ. Subrahmanyan Chandrasekhar shared the 1983 Nobel Prize in Physics and had the Chandra X-ray Observatory named after him. Steven Ch

In [15]:
llm = InferenceClient(
        model=HF_HUB_MODELS["llama-2-7b"],
        token=HUGGINGFACEHUB_API_TOKEN,
    )

In [32]:
q_output = llm.text_generation(
        question_template.format(search_results=docs, question=q),
        max_new_tokens=200,
        temperature=1,
    )

In [27]:
reduce_template = """<s>[INST] <<SYS>>
Given the following extracted parts of a long document and a question, create a final answer.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

<</SYS>>

QUESTION: {question}
=========
{output}
=========
Answer:[/INST]"""

In [33]:
q_output

'  According to Document [1](Title: List of Nobel laureates in Physics), the first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen.'

In [34]:
output = llm.text_generation(
        reduce_template.format(output=q_output, question=q),
        max_new_tokens=100,
        temperature=1,
    )
output

'  Sure! According to Document [1], the first Nobel Prize in Physics was awarded to Wilhelm Conrad Röntgen in 1901. Therefore, the answer to the question is: Wilhelm Conrad Röntgen.'